In [1]:
#Import Libraries 
#import os 
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline 


In [2]:
daset=pd.read_csv('Google_Stock_Price_Train.csv')

In [3]:
daset.head()

,Date,Open,High,Low,Close,Volume
0,1/3/2012,325.25,332.83,324.97,663.59,"7,380,500"
1,1/4/2012,331.27,333.87,329.08,666.45,"5,749,400"
2,1/5/2012,329.83,330.75,326.89,657.21,"6,590,300"
3,1/6/2012,328.34,328.77,323.68,648.24,"5,405,900"
4,1/9/2012,322.04,322.29,309.46,620.76,"11,688,800"


In [9]:
# Use the Open Stock Price Column to Train Your Model
training_set=daset.iloc[:,1:2].values
print(training_set)
#how many how coloumns and rows in the data set
print(training_set.shape)

[[325.25]
 [331.27]
 [329.83]
 ...
 [793.7 ]
 [783.33]
 [782.75]]
(1258, 1)


In [10]:
#Normalization the dataset
from sklearn.preprocessing import MinMaxScaler
scaler =MinMaxScaler(feature_range=(0,1))
scaled_training_set = scaler.fit_transform(training_set)
scaled_training_set

array([[0.08581368],
       [0.09701243],
       [0.09433366],
       ...,
       [0.95725128],
       [0.93796041],
       [0.93688146]])

In [8]:
#Creating X_train and y_train
X_train= []
Y_train= []
for i in range(60,1258):
    X_train.append(scaled_training_set[i-60:i,0])
    Y_train.append(scaled_training_set[i,0])
X_train=np.array(X_train)
Y_train=np.array(Y_train)

NameError: name 'scaled_training_set' is not defined

In [ ]:
print(X_train.shape)
print(Y_train.shape)

In [ ]:
#Building the Model by Importing the Crucial Libraries and Adding Different Layers to LSTM
#import numpy
#import tensorflow.compat.v2 as tf
from keras.models import Sequential 
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Dropout

In [ ]:
regressor= Sequential()
regressor.add(LSTM(units=50,return_sequences=True,input_shape=(X_train.shape[1],1)))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units=50,return_sequences=True))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units=50,return_sequences=True))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units=50))
regressor.add(Dropout(0.2))

regressor.add(Dense(units=1))

In [ ]:
#fitting the model 
regressor.compile(optimizer='adam',loss='mean_squared_error')
regressor.fit(X_train,Y_train,epochs=10,batch_size=32)



In [ ]:
#Extracting the Actual Stock Prices of Jan-2017.
dataset_test=pd.read_csv("Google_Stock_Price_Test.csv")
actual_stock_price=dataset_test.iloc[:,1:2].values

In [ ]:
 #Preparing the Input for the Model.
dataset_total=pd.concat((daset['Open'],dataset_test['Open']),axis=0)
inputs=dataset_total[len(dataset_total)-len(dataset_test)-60:].values
    
inputs=inputs.reshape(-1,1)
inputs=scaler.transform(inputs)
    
x_test=[]
for i in range(60,80):
    x_test.append(inputs[i-60:i,0])
x_test=np.array(x_test) 
x_test=np.reshape(x_test,(x_test.shape[0],x_test.shape[1],1))
    

In [ ]:
#Predicting the Values for Jan 2017 Stock Prices
predicted_stock_price=regressor.predict(x_test)
predicted_stock_price=scaler.inverse_transform(predicted_stock_price)


In [ ]:
 #Plotting the Actual and Predicted Prices for Google Stocks.
plt.plot(actual_stock_price,color='red',label='Actual Google Stock Price')
plt.plot(predicted_stock_price,color='blue',label='Predicted Google Stock Price')
plt.title('Google Stock Price Prediction')
plt.xlabel('Time')
plt.ylabel('Google Stock price')
plt.legend()